In [4]:
import cv2
import pandas as pd
import os
import shutil

# --- Cấu hình ---
BASE_VIDEO_PATH = "DATA/RawVideo" # Thư mục video gốc
CLIP_OUTPUT_PATH = "DATA/data_clips/train" # Thư mục lưu clip train
TRAIN_SPLIT_FILE = "train_data.txt" # File split

CLIP_DURATION_SEC = 5 # Độ dài mỗi clip (giây)
SLIDE_SEC = 2         # Trượt 2 giây (giảm chồng lấn, tăng tốc độ)

# --- (MỚI) Xây dựng bộ đệm (cache) tìm kiếm file ---
print(f"Đang quét thư mục video gốc: {BASE_VIDEO_PATH}...")
video_path_cache = {}
for root, dirs, files in os.walk(BASE_VIDEO_PATH):
    for file in files:
        if file.endswith(('.mp4', '.avi', '.x264')):
            video_path_cache[file] = os.path.join(root, file)
print(f"Đã tìm thấy {len(video_path_cache)} video.")
# ----------------------------------------------------



Đang quét thư mục video gốc: DATA/RawVideo...
Đã tìm thấy 1900 video.


In [5]:
# --- Hàm cắt video (cửa sổ trượt) ---
# (Hàm này giữ nguyên, không thay đổi)
def trim_video_sliding_window(source_video_name, label):
    
    if source_video_name not in video_path_cache:
        print(f"CẢNH BÁO (Train): Không tìm thấy video '{source_video_name}'. Bỏ qua.")
        return 0

    source_video_full_path = video_path_cache[source_video_name]
    
    label_dir = os.path.join(CLIP_OUTPUT_PATH, label)
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)

    cap = cv2.VideoCapture(source_video_full_path)
    if not cap.isOpened():
        print(f"LỖI: Không thể mở video gốc: {source_video_full_path}")
        return 0

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0: fps = 30
        
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    clip_duration_frames = int(fps * CLIP_DURATION_SEC)
    slide_frames = int(fps * SLIDE_SEC)
    
    clip_count = 0
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    for start_frame in range(0, total_frames - clip_duration_frames + 1, slide_frames):
        end_frame = start_frame + clip_duration_frames
        clip_count += 1
        output_clip_name = f"{source_video_name.split('.')[0]}_clip{clip_count}.mp4"
        output_clip_full_path = os.path.join(label_dir, output_clip_name)

        # Mở writer
        writer = cv2.VideoWriter(output_clip_full_path, fourcc, fps, (frame_width, frame_height))
        
        # Đặt vị trí frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        
        # Ghi N frame
        for i in range(clip_duration_frames):
            ret, frame = cap.read()
            if not ret: break
            writer.write(frame)
        
        writer.release()

    cap.release()
    # Tắt log này để đỡ nhiễu, chỉ bật khi cần debug
    print(f"  Đã cắt video '{source_video_name}' thành {clip_count} clips.")
    return clip_count



In [6]:
# --- Vòng lặp chính (ĐÃ CẬP NHẬT) ---
print("Bắt đầu chuẩn bị dữ liệu Train (Sliding Window)...")

# Đảm bảo thư mục gốc tồn tại
if not os.path.exists(CLIP_OUTPUT_PATH):
    os.makedirs(CLIP_OUTPUT_PATH)

try:
    with open(TRAIN_SPLIT_FILE, 'r') as f:
        train_videos_list = f.readlines()
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file split huấn luyện '{TRAIN_SPLIT_FILE}'.")
    exit()

total_clips_generated_this_run = 0
total_videos_processed = 0
total_videos_skipped = 0

print(f"Tổng số video cần kiểm tra: {len(train_videos_list)}")

for line in train_videos_list:
    line = line.strip()
    if not line: continue
    
    try:
        # Giả định định dạng là "Label/VideoName.mp4"
        label, video_name = line.split('/')
    except ValueError:
        print(f"Bỏ qua dòng không đúng định dạng: {line}")
        continue
    
    # Đặc biệt: Xử lý cho "Normal" nếu nó nằm trong thư mục con khác
    if label == "Normal_Videos_event":
        label = "Normal"
        # (Tùy chỉnh nếu cần)
        
    # --- (THÊM MỚI) KIỂM TRA XEM ĐÃ CẮT CHƯA ---
    # Kiểm tra sự tồn tại của clip đầu tiên
    label_dir = os.path.join(CLIP_OUTPUT_PATH, label)
    first_clip_name = f"{video_name.split('.')[0]}_clip1.mp4"
    first_clip_path = os.path.join(label_dir, first_clip_name)
    
    if os.path.exists(first_clip_path):
        # Nếu file clip 1 đã tồn tại, ta giả định video này đã được xử lý
        total_videos_skipped += 1
        continue
    # -------------------------------------------

    # Nếu chưa tồn tại, tiến hành cắt
    print(f"Đang xử lý (cắt mới): '{video_name}'...")
    num_clips = trim_video_sliding_window(video_name, label)
    total_clips_generated_this_run += num_clips
    total_videos_processed += 1

print(f"\nHoàn thành!")
print(f"  Đã xử lý (cắt mới) trong lần chạy này: {total_videos_processed} video (tạo ra {total_clips_generated_this_run} clips).")
print(f"  Đã bỏ qua (vì đã có sẵn): {total_videos_skipped} video.")

Bắt đầu chuẩn bị dữ liệu Train (Sliding Window)...
Tổng số video cần kiểm tra: 532
Đang xử lý (cắt mới): 'Abuse013_x264.mp4'...
  Đã cắt video 'Abuse013_x264.mp4' thành 79 clips.
Đang xử lý (cắt mới): 'Abuse014_x264.mp4'...
  Đã cắt video 'Abuse014_x264.mp4' thành 41 clips.
Đang xử lý (cắt mới): 'Abuse015_x264.mp4'...
  Đã cắt video 'Abuse015_x264.mp4' thành 129 clips.
Đang xử lý (cắt mới): 'Abuse016_x264.mp4'...
  Đã cắt video 'Abuse016_x264.mp4' thành 21 clips.
Đang xử lý (cắt mới): 'Abuse017_x264.mp4'...
  Đã cắt video 'Abuse017_x264.mp4' thành 30 clips.
Đang xử lý (cắt mới): 'Abuse018_x264.mp4'...
  Đã cắt video 'Abuse018_x264.mp4' thành 38 clips.
Đang xử lý (cắt mới): 'Abuse019_x264.mp4'...
  Đã cắt video 'Abuse019_x264.mp4' thành 64 clips.
Đang xử lý (cắt mới): 'Abuse020_x264.mp4'...
  Đã cắt video 'Abuse020_x264.mp4' thành 50 clips.
Đang xử lý (cắt mới): 'Abuse021_x264.mp4'...
  Đã cắt video 'Abuse021_x264.mp4' thành 30 clips.
Đang xử lý (cắt mới): 'Abuse022_x264.mp4'...
  Đã cắ